# Т-Банк: отели и авиабилеты — EDA
**Файл:** dano_dataset_travel.csv` **Период:** 2024-01-01 — 2024-11-07

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt

df = pd.read_csv("dano_dataset_travel.csv", sep=";", encoding="utf-8", low_memory=False)


DATE_COLS = ['party_first_order_dt', 'party_first_order_type_dt', 'free_cancel_booking_dttm', 'created_dttm', 'cancel_dttm', 'book_start_dttm', 'local_book_start_dttm', 'book_end_dttm', 'last_sms_dt', 'last_email_send_dt', 'last_session_dttm']
for c in DATE_COLS:
    if c in df.columns: df[c] = pd.to_datetime(df[c], errors="coerce")

NUM_COLS = ['promo_code_discount_amt', 'loyalty_accrual_rub_amt', 'nominal_price_eur_amt', 'nominal_price_rub_amt', 'monthly_income_amt']
def coerce_numeric(s):
    return pd.to_numeric(s.astype(str).str.replace(" ","").str.replace("\xa0","").str.replace(",", "."),
                         errors="coerce")
for c in NUM_COLS:
    if c in df.columns: df[c] = coerce_numeric(df[c])


FLAG_COLS = [c for c in df.columns if c.endswith("_flg")]
for c in FLAG_COLS:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")
    df[c] = (df[c].fillna(0) > 0)


df["order_type"] = df.get("order_type_cd").map({"AIR":"Авиабилеты","HOT":"Отели"}).fillna(df.get("order_type_cd"))
df["is_success"] = df.get("order_status_cd","").eq("SUC")
df["order_month"] = df["created_dttm"].dt.to_period("M").astype(str)


df["lead_time_days"] = (df["book_start_dttm"] - df["created_dttm"]).dt.days if "book_start_dttm" in df.columns else np.nan
if "book_end_dttm" in df.columns and "book_start_dttm" in df.columns:
    df["stay_nights"] = (df["book_end_dttm"] - df["book_start_dttm"]).dt.days
    df.loc[df.get("order_type_cd")!="HOT", "stay_nights"] = np.nan

df.info()


## Сводка и графики

In [ ]:
df.groupby("order_type").agg(orders=("order_rk","count"), success_rate=("is_success","mean"), avg_price_rub=("nominal_price_rub_amt","mean"))

In [ ]:
df.groupby(["order_month","order_type"])["order_rk"].count().unstack(fill_value=0).plot(marker="o"); plt.title("Динамика заказов по месяцам"); plt.xticks(rotation=45); plt.tight_layout()

In [ ]:
(df.groupby("order_type")["is_success"].mean()*100).plot(kind="bar"); plt.title("SUC-rate, %"); plt.tight_layout()

## Гипотезы
- Показывать промо-коды пользователям с коротким лид-таймом (≤3 дня) → рост конверсии last-minute.
- Упростить оплату до one-click для лояльных (bundle Pro/Premium, участники LP) → рост SUC-rate, меньше дропа.
- Показывать пакеты «авиабилет+отель» в поиске/корзине → рост AOV и кросс-продаж.

**Выбор:** Показывать пакеты «авиабилет+отель» в поиске/корзине → рост AOV и кросс-продаж.